In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/shelfmate.db')
c = con.cursor()

# Creating DB Schema

In [3]:
c.execute("PRAGMA foreign_keys = ON;")

In [4]:
c.execute('''
          CREATE TABLE users
          (username TEXT PRIMARY KEY,
          password TEXT NOT NULL,
          name TEXT NOT NULL,
          gender CHECK(gender IN ('Male', 'Female', 'Other')),
          email TEXT NOT NULL,
          district CHECK(district IN ("Aveiro",
                                    "Beja",
                                    "Braga",
                                    "Bragança",
                                    "Castelo Branco",
                                    "Coimbra",
                                    "Évora",
                                    "Faro",
                                    "Guarda",
                                    "Leiria",
                                    "Lisboa",
                                    "Portalegre",
                                    "Porto",
                                    "Santarém",
                                    "Setúbal",
                                    "Viana do Castelo",
                                    "Vila Real",
                                    "Viseu",
                                    "Açores",
                                    "Madeira"))) WITHOUT ROWID
        ''')

In [5]:
c.execute('''
          CREATE TABLE books
          (book_id INTEGER PRIMARY KEY,
          title TEXT NOT NULL,
          description TEXT NOT NULL,
          published_date INTEGER,
          page_number INTEGER NOT NULL,
          rating REAL) WITHOUT ROWID
          ''')

In [6]:
c.execute('''
          CREATE TABLE genres
          (genre_id INTEGER PRIMARY KEY,
          genre TEXT NOT NULL) WITHOUT ROWID
          ''')

In [7]:
c.execute('''
          CREATE TABLE authors
          (author_id INTEGER PRIMARY KEY,
          author_name TEXT NOT NULL,
          top_genre INTEGER NOT NULL,
          FOREIGN KEY (top_genre) REFERENCES genres(genre_id)) WITHOUT ROWID
          ''')

In [8]:
c.execute('''
          CREATE TABLE authors_books
          (book_id INTEGER NOT NULL,
          author_id INTEGER NOT NULL,
          FOREIGN KEY (book_id) REFERENCES books(book_id),
          FOREIGN KEY (author_id) REFERENCES authors(author_id))
          ''')

In [9]:
c.execute('''
          CREATE TABLE read_list
          (username TEXT NOT NULL,
          book_id INTEGER NOT NULL,
          rating INTEGER CHECK (rating IS NULL OR (rating >= 1 AND rating <= 5)),
          did_not_finish_flag BINARY NOT NULL DEFAULT 0,
          FOREIGN KEY (username) REFERENCES users(username),
          FOREIGN KEY (book_id) REFERENCES books(book_id))
          ''')

In [10]:
c.execute('''
          CREATE TABLE fav_authors
          (username TEXT NOT NULL,
          author_id INTEGER NOT NULL,
          FOREIGN KEY (username) REFERENCES users(username),
          FOREIGN KEY (author_id) REFERENCES authors(author_id))
          ''')

In [11]:
c.execute('''
          CREATE TABLE books_genres
          (book_id INTEGER NOT NULL,
          genre_id INTEGER NOT NULL,
          FOREIGN KEY (book_id) REFERENCES books(book_id),
          FOREIGN KEY (genre_id) REFERENCES genres(genre_id))
          ''')

In [12]:
c.execute('''
          CREATE TABLE fav_genres
          (username TEXT NOT NULL,
          genre_id INTEGER NOT NULL,
          FOREIGN KEY (username) REFERENCES users(username),
          FOREIGN KEY (genre_id) REFERENCES genres(genre_id))
          ''')

In [13]:
con.commit()
con.close()

# Populating the DB

In [14]:
con = sqlite3.connect('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/shelfmate.db')
c = con.cursor()

## Books Table

In [15]:
books = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/books.csv')
books.head()

,Title,description,authors,publishedDate,categories,number_pages,book_id,review/score
0,History of Magic and the Occult,"See the history of witchcraft, magic and super...",['DK'],2020,"['Body, Mind & Spirit']",293,1,4.000000
1,By Honor Bound (The Lassiter Law) (Silhouette ...,A comprehensive guide that defines the literat...,['Kristin Ramsdell'],2012,['Language Arts & Disciplines'],272,2,4.000000
2,Usa Laminated Map,"The United States ""Explorer"" map is a classic ...",['National Geographic Maps'],2015,['Reference'],237,3,5.000000
3,America at 1750: A Social Portrait,Demonstrates how the colonies developed into t...,['Richard Hofstadter'],1973,['History'],320,4,3.777778
4,How to Discipline Kids without Losing Their Lo...,Imagine... No More Arguing. Imagine... No More...,['Jim Fay'],2004,['Family & Relationships'],180,5,4.882353


In [16]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19294 entries, 0 to 19293
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          19294 non-null  object 
 1   description    19294 non-null  object 
 2   authors        19294 non-null  object 
 3   publishedDate  19294 non-null  int64  
 4   categories     19294 non-null  object 
 5   number_pages   19294 non-null  int64  
 6   book_id        19294 non-null  int64  
 7   review/score   19294 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 1.2+ MB


In [19]:
for b in books.book_id:
    row = books[books['book_id'] == b].iloc[0] 

    values = (
        float(row['book_id']),
        row['Title'],
        row['description'],
        float(row['publishedDate']),
        float(row['number_pages']),
        float(row['review/score']))
    
    c.execute("INSERT INTO books (book_id, title, description, published_date, page_number, rating) VALUES (?,?,?,?,?,?)", values)
con.commit()

## Users Table

In [20]:
users = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/final_users.csv')

In [21]:
users.head()

,username,name,email,password,gender,district
0,sunny22,Sandra Silva,sandra.silva@example.com,Sunshine123,female,Lisbon
1,techwizard,Tomás Teixeira,tomas.teixeira@example.com,Techie456,male,Porto
2,flowerpower,Fátima Fernandes,fatima.fernandes@example.com,Floral789,female,Faro
3,gamer123,Gonçalo Gomes,goncalo.gomes@example.com,GameOn321,male,Braga
4,bookworm,Beatriz Barbosa,beatriz.barbosa@example.com,ReadMore987,female,Aveiro


In [22]:
# for u in users.username:
#     row = users[users['username'] == u].iloc[0] 

#     values = (
#         row['username'],
#         row['name'],
#         row['email'],
#         row['password'],
#         row['gender'],
#         row['district'])
    
#     c.execute("INSERT INTO users (username, name, email, password, gender, district) VALUES (?,?,?,?,?,?)", values)
# con.commit()

In [23]:
# Constraints working

In [24]:
users['gender'] = users.gender.str.title()

In [25]:
users.iloc[0,-1] = 'Lisboa'

In [26]:
for u in users.username:
    row = users[users['username'] == u].iloc[0] 

    values = (
        row['username'],
        row['name'],
        row['email'],
        row['password'],
        row['gender'],
        row['district'])
    
    c.execute("INSERT INTO users (username, name, email, password, gender, district) VALUES (?,?,?,?,?,?)", values)
con.commit()

## fav_genres Table

In [27]:
fav_genres = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/fav_genres.csv')
fav_genres.head()

,username,genre_id
0,sunny22,7
1,sunny22,15
2,techwizard,3
3,techwizard,18
4,techwizard,22


In [28]:
for i in fav_genres.index:
    row = fav_genres.iloc[i] 

    values = (
        row['username'],
        float(row['genre_id']))
    
    c.execute("INSERT INTO fav_genres (username, genre_id) VALUES (?,?)", values)
con.commit()

## fav_authors table

In [29]:
fav_authors = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/fav_authors.csv')
fav_authors.head()

,username,author_id
0,sunny22,4021
1,sunny22,5185
2,sunny22,2865
3,techwizard,1564
4,techwizard,3992


In [30]:
for i in fav_authors.index:
    row = fav_authors.iloc[i] 

    values = (
        row['username'],
        float(row['author_id']))
    
    c.execute("INSERT INTO fav_authors (username, author_id) VALUES (?,?)", values)
con.commit()

## Authors Table

In [31]:
authors = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/authors.csv')
authors.head()

,author_id,author,genre_id
0,1,DK,7
1,2,Kristin Ramsdell,2
2,3,National Geographic Maps,3
3,4,Richard Hofstadter,4
4,5,Jim Fay,5


In [32]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5751 entries, 0 to 5750
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  5751 non-null   int64 
 1   author     5751 non-null   object
 2   genre_id   5751 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 134.9+ KB


In [33]:
for a in authors.author_id:
    row = authors[authors['author_id'] == a].iloc[0] 

    values = (
        float(row['author_id']),
        row['author'],
        float(row['genre_id']))
    
    c.execute("INSERT INTO authors (author_id, author_name, top_genre) VALUES (?,?,?)", values)
con.commit()

## Read List Table

In [34]:
read_list = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/read_list.csv')
read_list.head()

,username,book_id,rating,did_not_finish_flag
0,sunny22,13456,5.0,0
1,sunny22,289,3.0,0
2,sunny22,7567,NaN,1
3,sunny22,16789,2.0,0
4,sunny22,9087,4.0,0


In [35]:
for i in read_list.index:
    row = read_list.iloc[i] 

    values = (
        row['username'],
        float(row['book_id']),
        float(row['rating']),
        float(row['did_not_finish_flag']))
    
    c.execute("INSERT INTO read_list (username, book_id, rating, did_not_finish_flag) VALUES (?,?,?,?)", values)
con.commit()

## Genres Table

In [36]:
genres = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/genres.csv')
genres.head()

,genre_id,genre
0,1,"Body, Mind & Spirit"
1,2,Language Arts & Disciplines
2,3,Reference
3,4,History
4,5,Family & Relationships


In [37]:
for i in genres.genre_id:
    row = genres[genres['genre_id'] == i].iloc[0] 

    values = (
        float(row['genre_id']),
        row['genre'])
    
    c.execute("INSERT INTO genres (genre_id, genre) VALUES (?,?)", values)
con.commit()

## authors_books Table

In [38]:
authors_books = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/authors_books.csv')
authors_books.head()

,book_id,author_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [39]:
for i in authors_books.index:
    row = authors_books.iloc[i] 

    values = (
        float(row['book_id']),
        float(row['author_id']))
    
    c.execute("INSERT INTO authors_books (book_id, author_id) VALUES (?,?)", values)
con.commit()

## books_genres Table

In [40]:
books_genres = pd.read_csv('C:/Users/mnasc/Desktop/LCD/3rd Year/Capstone Project/Project/data/final_data/books_genres.csv')
books_genres.head()

,book_id,genre_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [41]:
for i in books_genres.index:
    row = books_genres.iloc[i] 

    values = (
        float(row['book_id']),
        float(row['genre_id']))
    
    c.execute("INSERT INTO books_genres (book_id, genre_id) VALUES (?,?)", values)
con.commit()

In [42]:
con.close()